# Highway network skeletonization

@anastassiavybornova : todo to make reproducible & fit with rest of workflow: 
* add bleeding edge pip install of uscuni/simplification to env
* yml with settings for 3 different cities
* loop over CBSAs
* refactor directories to fit this repo

Jacksonville: 27260, Milwaukee: 33340, Portland: 38900

Compare with: https://www.dropbox.com/scl/fo/y5ztboz4lnxjhhedrjwx5/AFMMecr0XnktH_ZlTnkGaIc?rlkey=ocz06xoeh0bp1cc69jchgn954&dl=0

12060_Atlanta_map.shp


In [ ]:
import logging
import warnings

import folium
from core import algorithms, utils
import geopandas as gpd
import osmnx as ox
import shapely
import numpy as np
import pandas as pd
from shapely.geometry import LineString
from core.geometry import voronoi_skeleton
import momepy
import networkx as nx

Filter out the RuntimeWarning showing on Apple Silicon.

In [ ]:
warnings.filterwarnings(
    "ignore",
    category=RuntimeWarning,
    message="invalid value encountered in intersection",
)

Set logging level to debug to see the debugging messages.

In [ ]:
# # Get the logger for core.algorithms.simplify
# logger = logging.getLogger('core.algorithms.simplify')
# logger.setLevel(logging.DEBUG)

# # Set the logging format
# formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# # Create a handler for the logger
# handler = logging.StreamHandler()
# handler.setLevel(logging.DEBUG)
# handler.setFormatter(formatter)

# # Add the handler to the logger
# logger.addHandler(handler)

In [ ]:
# specify case metadata (TODO: loop over, same parameters for all 3 cities?)
case = 38900

# SETTINGS JUST FOR PORTLAND
mybuffer = 75 # TODO: set to 300 for other 2 cities!!
length_threshold = 0.1*10**4 # TODO: set to 10**4 for other 2 cities!!
my_coordnum = 30
my_maxseglen = 50
my_danglefactor = 10 # TODO: set to 2.5 for other 2 cities!

In [ ]:
def load_graph(mycbsacode):

    lcc = ox.load_graphml(
        f"../data/{mycbsacode}/lcc.graphml",
        edge_dtypes={
            "length": float,
        },
    )
    # get its coordinate system
    crs_byox = lcc.graph["crs"]

    # convert street network to gdf
    network_gdf = ox.graph_to_gdfs(
            G = lcc, 
            nodes = False, 
            edges = True, 
            node_geometry = False, 
            fill_edge_geometry = True)

    # set crs
    network_gdf = network_gdf.set_crs(crs_byox)

    return network_gdf

In [ ]:
# specify projection
proj_crs = "epsg:9311"

# read road data
roads = load_graph(case)

# project (so that we can buffer)
roads = roads.to_crs(proj_crs)

# buffer poly from all roads
geom = roads.buffer(mybuffer).union_all()
ser = gpd.GeoSeries([geom], crs = roads.crs)
bou = ser.boundary
bou = bou.explode().reset_index(drop=True)
bou = gpd.GeoDataFrame({"geometry":bou}, crs = roads.crs)

In [ ]:
# check if correct
m = bou[bou.length>length_threshold].explore(tiles = "cartodb positron", name = "bou")
roads.explore(m=m, color = "black")
folium.LayerControl().add_to(m)
m

In [ ]:
# drop too-short linestrings
bou_red = bou[bou.length>length_threshold].copy()

# polygonize
poly = shapely.polygonize(
    np.array(
        bou_red.geometry
    )
)

gdf = gpd.GeoDataFrame(
    {
        "geometry": poly.geoms
    },
    crs = roads.crs
)

gdf["i"] = gdf.index

In [ ]:
# which one is the highway polygon?
highway_index = 0

In [ ]:
m = roads.explore(max_zoom=52, tiles="cartodb positron", color="black", prefer_canvas=True, name = "roads")
gdf.explore(m=m, name = "poly", column = "i", opacity = .8)
gdf[gdf["i"]==highway_index].explore(m=m, name="highway", color = "red")
folium.LayerControl().add_to(m)
m

In [ ]:
highway = gdf.geometry[highway_index]

In [ ]:
delin = gpd.GeoDataFrame(
    {
        "geometry":
        [
            LineString(r) for r in highway.interiors] + [LineString(highway.exterior)
        ]
    },
    crs = roads.crs
)
delin

In [ ]:
def unzip_line(geom, coordnum = 30):
    longline = [c for c in geom.coords]
    linestrings = []
    current_linestring = []
    for c in longline:
        current_linestring.append(c)
        if len(current_linestring) > coordnum:
            linestrings.append(LineString(current_linestring))
            del current_linestring
            current_linestring = [c]
    if current_linestring:
        linestrings.append(LineString(current_linestring))
    return linestrings

In [ ]:
all_lines = []

for geom in delin.geometry:
    all_lines+= (
        unzip_line(geom, my_coordnum)
    )

In [ ]:
lines = gpd.GeoDataFrame({"geometry": all_lines}, crs = roads.crs)
lines["i"] = lines.index
lines.explore(tiles="cartodb positron", column="i")

In [ ]:
skel = voronoi_skeleton(
    lines = lines.geometry,
    poly = highway,
    max_segment_length = my_maxseglen
)

In [ ]:
m = roads.explore(max_zoom=52, tiles="cartodb positron", color="black", prefer_canvas=True, name = "roads")
lines.explore(m=m, column="i", name ="lines")
gdf[gdf["i"]==0].explore(m=m,color="yellow", name = "highway", opacity = .7)
gpd.GeoSeries(skel[0], crs = roads.crs).explore(m=m, name = "skel", color = "red", opacity = 0.9, style_kwds={"weight":2})
folium.LayerControl().add_to(m)
m

In [ ]:
skel_gdf = gpd.GeoDataFrame({"geometry":skel[0]}, crs = roads.crs)
skel_gdf["len"] = skel_gdf.length

In [ ]:
G = momepy.gdf_to_nx(gdf_network=skel_gdf, multigraph = False, integer_labels=True)
for _ in range(5):
    degree_one = [n for n in G.nodes if nx.degree(G, n)==1]
    degree_one_to_remove = []
    for n in degree_one:
        if G.edges[list(G.edges(n))[0]]["mm_len"] < my_danglefactor * mybuffer:
            degree_one_to_remove.append(n)
    G.remove_nodes_from(degree_one_to_remove)
edges = momepy.nx_to_gdf(G, points=False, lines=True)

In [ ]:
edges_rem = momepy.remove_false_nodes(edges)
edges_rem["i"] = edges_rem.index

## Check results:

In [ ]:
m = roads.explore(max_zoom=52, tiles="cartodb positron", color="black", prefer_canvas=True, name = "roads")
lines.explore(m=m, column="i", name ="lines")
#gdf[gdf["i"]==0].explore(m=m,color="yellow", name = "highway", opacity = .7)
gpd.GeoSeries(skel[0], crs = roads.crs).explore(m=m, name = "skel", color = "red", opacity = 0.9, style_kwds={"weight":2})
edges.explore(m=m, name = "edges", color = "green", style_kwds={"weight":3})
edges_rem.explore(m=m, name = "edges_rem", column = "i", style_kwds={"weight":3}, highlight_kwds={"color":"red"})
folium.LayerControl().add_to(m)
m

# Save results

In [ ]:
edges_rem = edges_rem.to_crs("EPSG:4326")
edges_rem[["geometry"]].to_file(f"../data/{case}/{case}_simplified.gpkg", index = False)